In [1]:
import sys
sys.path.append("..")
sys.path.append("../lmmnn/")

from utils.training_functions import *
from utils.evaluation import *
from utils.utils import tune_xgboost,tune_lasso,evaluate_lr,evaluate_xgb

from data import dataset_preprocessing

from utils.evaluation import get_metrics
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso

from scipy import stats


import pickle

In [107]:
dataset_name = "cortez"
mode="cv"
RS=1
hct=10
test_ratio=0.2
val_ratio=0.1
folds=5
target = "continuous"
experiment_name = "5CV_paper_final"

### Describe raw data

In [3]:
import arff # make sure to pip install liac-arff



df_mat = pd.read_csv(f"../data/raw/{dataset_name}/student-mat.csv",sep=";")
df_por = pd.read_csv(f"../data/raw/{dataset_name}/student-por.csv",sep=";")

df_mat["subject"] = "mat"
df_por["subject"] = "por"

df = pd.concat([df_mat,df_por])

In [76]:
y_col = "G3"
demographic_cols = ["sex", "age", "school", "address", "famsize", "famrel", "Pstatus", "Medu", "Fedu", "Mjob", "Fjob","reason","guardian", "traveltime", "famsup","paid","internet"]
perf_cols = ["G1","G2", "failures"]
activity_cols = ["studytime", "schoolsup", 'activities', 'nursery', 'higher', 'absences']
other_cols = ["subject", "Dalc", "Walc", "health", 'freetime', 'goout', 'romantic']
cat_cols = list(set(df.columns[np.logical_and(df.dtypes == "object", df.nunique()>2)]) - set([y_col]))

set(df.columns)-set([y_col]+demographic_cols+perf_cols+activity_cols+other_cols)

set()

In [6]:
desc_df_dict = {"N": df.shape[0],
           "d": df.shape[1],
           "% NA": df.isna().sum().sum()/sum(df.shape),
           "Target": f"$y \in ${[1, df[y_col].nunique()]}",
           "Performance features": len(perf_cols),
           "Demographic features": len(demographic_cols),
           "Activity features": len(activity_cols),
           "Other features": len(other_cols),
           "Categorical features": len(df.columns[list(np.logical_and(df.nunique() > 2, df.dtypes == "object"))]),     
           "Total cardinality": df[df.columns[list(np.logical_and(df.nunique() > 2, df.dtypes == "object"))]].nunique().sum(),     
#            "High cardinality levels":  list(df.loc[:,list(df.columns[list(np.logical_and(df.nunique() >= 10, df.dtypes == "object"))])].nunique().sort_values().values),
          
}
desc_df = pd.DataFrame([desc_df_dict],index=["cortez"])
desc_df

,N,d,% NA,Target,Performance features,Demographic features,Activity features,Other features,Categorical features,Total cardinality
cortez,1044,34,0.0,"$y \in $[1, 19]",2,17,7,7,4,17


In [7]:
pd.DataFrame(pd.concat([df.drop(y_col,axis=1),pd.Series(df[y_col].values,index=df.index,name="target")],axis=1).corr()["target"])

,target
age,-0.125282
Medu,0.201472
Fedu,0.159796
traveltime,-0.102627
studytime,0.161629
failures,-0.383145
famrel,0.054461
freetime,-0.064890
goout,-0.097877
Dalc,-0.129642


In [8]:
print(desc_df.transpose().to_latex())

\begin{tabular}{ll}
\toprule
{} &           cortez \\
\midrule
N                    &             1044 \\
d                    &               34 \\
\% NA                 &              0.0 \\
Target               &  \$y \textbackslash in \$[1, 19] \\
Performance features &                2 \\
Demographic features &               17 \\
Activity features    &                7 \\
Other features       &                7 \\
Categorical features &                4 \\
Total cardinality    &               17 \\
\bottomrule
\end{tabular}



### Preprocessing and preparation

In [9]:
data_path = f"{mode}_RS{RS}_hct{hct}"
if mode == "cv":
    data_path += f"_{folds}folds"
elif mode == "train_test":
    data_path += f"_split{1-test_ratio*100}-{test_ratio*100}"
elif mode == "train_val_test":
    data_path += f"_split{round(100-(test_ratio+val_ratio)*100)}-{round(test_ratio*100)}-{round(val_ratio*100)}"


# If no data_dict for the configuration exists, run preprocessing, else load data_dict
if not os.path.exists(f"../data/prepared/{dataset_name}/"+data_path+"/data_dict.pickle"):
    dataset_preprocessing.process_dataset(dataset_name, target, mode, RS, hct, test_ratio, val_ratio, folds)
with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
        data_dict = pickle.load(handle)


## Evaluation of categorical data treatment methods

In [10]:
conditions = ["ignore", "ohe", "target", "ordinal", "catboost", "glmm"]

In [108]:
early_stopping_rounds = 10
max_evals = 50

In [109]:
if not os.path.exists(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle"):

    results_encodings = {}
    results_encodings_feature_importances = {}

    for fold in range(folds):
        results_encodings[fold] = {}
        results_encodings_feature_importances[fold] = {}
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = data_dict[f"y_test_{fold}"]
        y_train_val = np.concatenate([y_train,y_val])

        y_train_val_pred_base = np.zeros(y_train_val.shape[0])#*np.mean(y_train_val)
        y_test_pred_base = np.zeros(y_test.shape[0])#*np.mean(y_train_val)

        results_encodings[fold]["Baseline"] = {}
        eval_res_train = get_metrics(y_train_val, y_train_val_pred_base, target=target)
        for metric in eval_res_train.keys():
            results_encodings[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(y_test, y_test_pred_base, target=target)
        for metric in eval_res_test.keys():
            results_encodings[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]


        for condition in conditions:
            print(f"Preparing results for fold {fold}, condition={condition}")
            # Retrieve data
            z_cols = data_dict["z_cols"]

            X_train = data_dict[f"X_train_{fold}"]
            y_train = data_dict[f"y_train_{fold}"]

            X_val = data_dict[f"X_val_{fold}"]
            y_val = data_dict[f"y_val_{fold}"]

            X_test = data_dict[f"X_test_{fold}"]
            y_test = data_dict[f"y_test_{fold}"]

    ## ALL BUT PERFORMANCE:
            # Define data subset for evaluation
    #         X_train = X_train[[i for i in X_train.columns if all([j not in i for j in perf_cols])]]
    #         X_val = X_val[[i for i in X_val.columns if all([j not in i for j in perf_cols])]]
    #         X_test = X_test[[i for i in X_test.columns if all([j not in i for j in perf_cols])]]

            # Define condition data subset
    #         if condition != "ignore":
    #             z_encoded_train = data_dict[f"z_{condition}_encoded_train_{fold}"] 
    #             z_encoded_val = data_dict[f"z_{condition}_encoded_val_{fold}"] 
    #             z_encoded_test = data_dict[f"z_{condition}_encoded_test_{fold}"] 

    #             X_train = pd.concat([X_train,z_encoded_train],axis=1)
    #             X_val = pd.concat([X_val,z_encoded_val],axis=1)
    #             X_test = pd.concat([X_test,z_encoded_test],axis=1)

    # ALL BUT PERFORMANCE & ACTIVITY:
    #         Define data subset for evaluation
            X_train = X_train[[i for i in X_train.columns if all([j not in i for j in perf_cols+activity_cols])]]
            X_val = X_val[[i for i in X_val.columns if all([j not in i for j in perf_cols+activity_cols])]]
            X_test = X_test[[i for i in X_test.columns if all([j not in i for j in perf_cols+activity_cols])]]

    #         Define condition data subset
            if condition != "ignore":
                z_encoded_train = data_dict[f"z_{condition}_encoded_train_{fold}"] 
                z_encoded_val = data_dict[f"z_{condition}_encoded_val_{fold}"] 
                z_encoded_test = data_dict[f"z_{condition}_encoded_test_{fold}"] 

                X_train = pd.concat([X_train,z_encoded_train],axis=1)
                X_val = pd.concat([X_val,z_encoded_val],axis=1)
                X_test = pd.concat([X_test,z_encoded_test],axis=1)


    ## ONLY CATEGORICAL: --> Produces trash as almost never better than baseline
    #         if condition != "ignore":        
    #             X_train = data_dict[f"z_{condition}_encoded_train_{fold}"] 
    #             X_val = data_dict[f"z_{condition}_encoded_val_{fold}"] 
    #             X_test = data_dict[f"z_{condition}_encoded_test_{fold}"] 

    #         else:
    #             continue

            X_train_val = pd.concat([X_train,X_val])
            y_train_val = np.concatenate([y_train,y_val])

            # Train base models
            res, feats = evaluate_lr(X_train_val, y_train_val, X_test, y_test, target=target,tune=False, seed=RS)
            results_encodings[fold]["LR_"+condition] = res
            results_encodings_feature_importances[fold]["LR_"+condition] = feats

            res, feats = evaluate_xgb(X_train_val, y_train_val, X_test, y_test, target, tune=False, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_encodings[fold]["XGB_"+condition] = res
            results_encodings_feature_importances[fold]["XGB_"+condition] = feats

            # Train tuned models
            res, feats = evaluate_lr(X_train_val, y_train_val, X_test, y_test, target=target, max_evals=max_evals, tune=True, seed=RS)
            results_encodings[fold]["LR_"+condition+"_tuned"] = res
            results_encodings_feature_importances[fold]["LR_"+condition+"_tuned"] = feats

            res, feats = evaluate_xgb(X_train_val, y_train_val, X_test, y_test, target, tune=True, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_encodings[fold]["XGB_"+condition+"_tuned"] = res
            results_encodings_feature_importances[fold]["XGB_"+condition+"_tuned"] = feats
    
    if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
        os.makedirs(f"../results/{dataset_name}/{experiment_name}")
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'wb') as handle:
        pickle.dump(results_encodings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings_feature_importances.pickle", 'wb') as handle:
        pickle.dump(results_encodings_feature_importances, handle, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'rb') as handle:
        results_encodings = pickle.load(handle)
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings_feature_importances.pickle", 'rb') as handle:
        results_encodings_feature_importances = pickle.load(handle)
        
        
results_encodings_df = pd.DataFrame(results_encodings[0]).transpose().sort_values("MSE Test",ascending=False).round(4)
results_encodings_df[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

Preparing results for fold 0, condition=ignore
SCORE: 0.9355610609054565                                                                                              
SCORE: 0.8965511322021484                                                                                              
SCORE: 1.0072535276412964                                                                                              
SCORE: 0.9777640104293823                                                                                              
SCORE: 1.0072535276412964                                                                                              
SCORE: 1.0072535276412964                                                                                              
SCORE: 1.0072535276412964                                                                                              
SCORE: 1.0072535276412964                                                                                        

SCORE: 0.8903347849845886                                                                                              
SCORE: 0.8863041996955872                                                                                              
SCORE: 0.8882527351379395                                                                                              
SCORE: 0.918246865272522                                                                                               
SCORE: 0.8769763708114624                                                                                              
SCORE: 1.1452094316482544                                                                                              
SCORE: 0.8993213772773743                                                                                              
SCORE: 0.8987678289413452                                                                                              
SCORE: 0.8960477709770203               

SCORE: 0.8351433873176575                                                                                              
SCORE: 0.8497704267501831                                                                                              
SCORE: 1.018872857093811                                                                                               
SCORE: 0.8407238125801086                                                                                              
SCORE: 0.8900095820426941                                                                                              
SCORE: 0.8425730466842651                                                                                              
SCORE: 0.9839725494384766                                                                                              
SCORE: 0.8332217335700989                                                                                              
SCORE: 0.842272162437439                

SCORE: 0.832933247089386                                                                                               
SCORE: 0.8407527208328247                                                                                              
SCORE: 0.8272923231124878                                                                                              
SCORE: 0.8335488438606262                                                                                              
SCORE: 0.8260032534599304                                                                                              
100%|████████████████████████████████████████████████| 50/50 [00:18<00:00,  2.77trial/s, best loss: 0.8100636601448059]
The best hyperparameters after step 3 are :  

{'learning_rate': 0.15489301925033427, 'n_estimators': 355.0, 'seed': 0, 'max_depth': 3.0, 'min_child_weight': 10.0, 'colsample_bytree': 0.8262525045137434, 'subsample': 0.8756460294555095}
Test Performance after third tuning round: 

SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 0.8713152259372048                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 0.9981252852519035                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0072535959884104               

...

SCORE: 1.049130139262597                                                                                               
SCORE: 1.049130139262597                                                                                               
SCORE: 0.9444401658383429                                                                                              
SCORE: 0.9274723860978101                                                                                              
SCORE: 1.0146482220597268                                                                                              
SCORE: 1.049130139262597                                                                                               
SCORE: 0.9253725123090953                                                                                              
SCORE: 1.0071565575423418                                                                                              
SCORE: 1.032541835090228                

SCORE: 0.9645215272903442                                                                                              
SCORE: 0.9602171182632446                                                                                              
SCORE: 0.9551604390144348                                                                                              
SCORE: 1.0344417095184326                                                                                              
SCORE: 0.9715967178344727                                                                                              
SCORE: 0.9606784582138062                                                                                              
SCORE: 0.953482449054718                                                                                               
SCORE: 0.9356397390365601                                                                                              
SCORE: 0.9548362493515015               

SCORE: 0.95036381483078                                                                                                
100%|████████████████████████████████████████████████| 50/50 [00:20<00:00,  2.44trial/s, best loss: 0.8967249989509583]
The best hyperparameters after step 2 are :  

{'learning_rate': 0.08374674097594241, 'n_estimators': 237.0, 'seed': 0, 'max_depth': 4.0, 'min_child_weight': 6.0}
Test Performance after second tuning round: 0.9587051868438721
SCORE: 0.9110112190246582                                                                                              
SCORE: 0.9233402013778687                                                                                              
SCORE: 0.9043012857437134                                                                                              
SCORE: 0.9022353291511536                                                                                              
SCORE: 0.917823314666748                              

SCORE: 1.0585567951202393                                                                                              
SCORE: 1.0362235307693481                                                                                              
SCORE: 1.0536361932754517                                                                                              
SCORE: 1.0758551359176636                                                                                              
SCORE: 1.1030384302139282                                                                                              
SCORE: 1.0805915594100952                                                                                              
SCORE: 0.990626335144043                                                                                               
SCORE: 1.067713975906372                                                                                               
SCORE: 1.0528781414031982               

,MSE Train,R2 Train,MSE Test,R2 Test
XGB_catboost,0.001000,0.999000,0.982500,-0.257000
XGB_ignore,0.008000,0.992000,0.950600,-0.216200
XGB_glmm,0.001100,0.998900,0.894800,-0.144800
XGB_target_tuned,0.110500,0.889600,0.893100,-0.142600
XGB_catboost_tuned,0.482000,0.518400,0.871200,-0.114600
XGB_ordinal,0.003900,0.996100,0.861200,-0.101800
XGB_ohe_tuned,0.354100,0.646200,0.855400,-0.094300
XGB_glmm_tuned,0.502500,0.497900,0.839300,-0.073800
XGB_target,0.004200,0.995800,0.821500,-0.051000
XGB_ordinal_tuned,0.338600,0.661700,0.804500,-0.029200


In [149]:
# experiment_name = "5CV_paper"
# if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
#     os.makedirs(f"../results/{dataset_name}/{experiment_name}")
# with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'wb') as handle:
#     pickle.dump(results_encodings, handle, protocol=pickle.HIGHEST_PROTOCOL)


### Feature Importances

In [37]:
# models = list(results_feature_importances[fold].keys())
# model = models[0]

# importances_df = pd.DataFrame(index=results_feature_importances[fold][model].index)
# for model in results_feature_importances[fold].keys():
#     importances_df.loc[:,model] = results_feature_importances[fold][model]
# importances_df

### Effectiveness of Parameter Tuning

In [110]:
models = results_encodings[0].keys()
metric = "MSE Test"

encodings_folds_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())
encodings_mean_df = encodings_folds_df.mean(axis=0)
encodings_std_df = encodings_folds_df.std(axis=0)

methods = sorted(list(encodings_mean_df.index))[1:]
not_tuned = ["Baseline"]+methods[::2]
tuned = ["Baseline"]+methods[1::2]

res_df_tune_comp_mean = pd.DataFrame([encodings_mean_df.loc[not_tuned].values,encodings_mean_df.loc[tuned]],index=["Untuned","Tuned"],columns=not_tuned).transpose()
res_df_tune_comp_std = pd.DataFrame([encodings_std_df.loc[not_tuned].values,encodings_std_df.loc[tuned]],index=["Untuned","Tuned"],columns=not_tuned).transpose()

res_df_tune_comp_mean.round(2).style.highlight_min(color = 'lightgreen', axis = 1)

,Untuned,Tuned
Baseline,1.010000,1.010000
LR_catboost,0.890000,0.880000
LR_glmm,0.880000,0.880000
LR_ignore,0.880000,0.880000
LR_ohe,0.890000,0.890000
LR_ordinal,0.890000,0.890000
LR_target,0.880000,0.880000
XGB_catboost,1.060000,0.930000
XGB_glmm,0.940000,0.900000
XGB_ignore,0.950000,0.890000


In [111]:
latex_df = res_df_tune_comp_mean.round(2).astype(str) + " (" +  res_df_tune_comp_std.round(3).astype(str) + ")"
latex_df

,Untuned,Tuned
Baseline,1.01 (0.175),1.01 (0.175)
LR_catboost,0.89 (0.115),0.88 (0.122)
LR_glmm,0.88 (0.119),0.88 (0.127)
LR_ignore,0.88 (0.121),0.88 (0.128)
LR_ohe,0.89 (0.113),0.89 (0.126)
LR_ordinal,0.89 (0.114),0.89 (0.119)
LR_target,0.88 (0.116),0.88 (0.116)
XGB_catboost,1.06 (0.119),0.93 (0.089)
XGB_glmm,0.94 (0.099),0.9 (0.073)
XGB_ignore,0.95 (0.09),0.89 (0.094)


In [112]:
print(latex_df.to_latex())

\begin{tabular}{lll}
\toprule
{} &       Untuned &         Tuned \\
\midrule
Baseline     &  1.01 (0.175) &  1.01 (0.175) \\
LR\_catboost  &  0.89 (0.115) &  0.88 (0.122) \\
LR\_glmm      &  0.88 (0.119) &  0.88 (0.127) \\
LR\_ignore    &  0.88 (0.121) &  0.88 (0.128) \\
LR\_ohe       &  0.89 (0.113) &  0.89 (0.126) \\
LR\_ordinal   &  0.89 (0.114) &  0.89 (0.119) \\
LR\_target    &  0.88 (0.116) &  0.88 (0.116) \\
XGB\_catboost &  1.06 (0.119) &  0.93 (0.089) \\
XGB\_glmm     &  0.94 (0.099) &   0.9 (0.073) \\
XGB\_ignore   &   0.95 (0.09) &  0.89 (0.094) \\
XGB\_ohe      &   0.9 (0.093) &  0.87 (0.102) \\
XGB\_ordinal  &  0.87 (0.058) &  0.89 (0.101) \\
XGB\_target   &  0.88 (0.071) &  0.89 (0.096) \\
\bottomrule
\end{tabular}



### Performance Comparison

In [113]:
# For LR
models = ["Baseline"]+[i for i in results_encodings[0].keys() if ("tuned" in i and "LR" in i)]
metric = "MSE Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())*-1

df_mean = pd.DataFrame((-1*use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_lr = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_lr.style.apply(negative_bold)


,Baseline,LR_ignore_tuned,LR_ohe_tuned,LR_target_tuned,LR_ordinal_tuned,LR_catboost_tuned,LR_glmm_tuned
0,1.01 (0.175),0.883 (0.128),0.887 (0.126),0.881 (0.116),0.89 (0.119),0.881 (0.122),0.883 (0.127)


In [114]:
# For LR
models = ["Baseline"]+[i for i in results_encodings[0].keys() if ("tuned" in i and "XGB" in i)]
metric = "MSE Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())*-1

df_mean = pd.DataFrame((-1*use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_xgb = pd.DataFrame([model_dict])
    
def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_xgb.style.apply(negative_bold)


,Baseline,XGB_ignore_tuned,XGB_ohe_tuned,XGB_target_tuned,XGB_ordinal_tuned,XGB_catboost_tuned,XGB_glmm_tuned
0,1.01 (0.175),0.887 (0.094),0.873 (0.102),0.886 (0.096),0.888 (0.101),0.93 (0.089),0.899 (0.073)


In [115]:
res_df_lr.columns = [i.split("_")[1] if i != "Baseline" else "Baseline" for i in res_df_lr.columns]    
res_df_xgb.columns = [i.split("_")[1] if i != "Baseline" else "Baseline" for i in res_df_xgb.columns]    

latex_df_encodings = pd.concat([res_df_lr,res_df_xgb],axis=0)
latex_df_encodings.index = ["LR", "XGB"]
latex_df_encodings

,Baseline,ignore,ohe,target,ordinal,catboost,glmm
LR,1.01 (0.175),0.883 (0.128),0.887 (0.126),0.881 (0.116),0.89 (0.119),0.881 (0.122),0.883 (0.127)
XGB,1.01 (0.175),0.887 (0.094),0.873 (0.102),0.886 (0.096),0.888 (0.101),0.93 (0.089),0.899 (0.073)


In [116]:
print(latex_df_encodings.round(2).to_latex())


\begin{tabular}{llllllll}
\toprule
{} &      Baseline &         ignore &            ohe &         target &        ordinal &       catboost &           glmm \\
\midrule
LR  &  1.01 (0.175) &  0.883 (0.128) &  0.887 (0.126) &  0.881 (0.116) &   0.89 (0.119) &  0.881 (0.122) &  0.883 (0.127) \\
XGB &  1.01 (0.175) &  0.887 (0.094) &  0.873 (0.102) &  0.886 (0.096) &  0.888 (0.101) &   0.93 (0.089) &  0.899 (0.073) \\
\bottomrule
\end{tabular}



### Data subset comparisons

As it does not matter which encoding method is used we use 5CV-GLMM encoding for LR and Ordinal encoding for XGB

In [401]:
early_stopping_rounds = 10
max_evals = 50

In [402]:
subsets = {"demo_only": demographic_cols,
           "performance_only": perf_cols,
           "activity_only": activity_cols,
           "activity_and_demo": activity_cols+demographic_cols,
           "performance_and_demo": perf_cols+demographic_cols,
           "all": list(df.columns)
          }

In [403]:
if not os.path.exists(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle"):

    results_subsets = {}
    results_subsets_feature_importances = {}

    for fold in range(folds):
        results_subsets[fold] = {}
        results_subsets_feature_importances[fold] = {}
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = data_dict[f"y_test_{fold}"]
        y_train_val = np.concatenate([y_train,y_val])

        y_train_val_pred_base = np.zeros(y_train_val.shape[0])#*np.mean(y_train_val)
        y_test_pred_base = np.zeros(y_test.shape[0])#*np.mean(y_train_val)

        results_subsets[fold]["Baseline"] = {}
        eval_res_train = get_metrics(y_train_val, y_train_val_pred_base, target=target)
        for metric in eval_res_train.keys():
            results_subsets[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(y_test, y_test_pred_base, target=target)
        for metric in eval_res_test.keys():
            results_subsets[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]


        for subset_key in subsets:
            print(f"Preparing results for fold {fold}, subset={subset_key}")
            # Retrieve data
            z_cols = data_dict["z_cols"]

            X_train = data_dict[f"X_train_{fold}"]
            y_train = data_dict[f"y_train_{fold}"]

            X_val = data_dict[f"X_val_{fold}"]
            y_val = data_dict[f"y_val_{fold}"]

            X_test = data_dict[f"X_test_{fold}"]
            y_test = data_dict[f"y_test_{fold}"]
        
            y_train_val = np.concatenate([y_train,y_val])

            # Define data subset for LR
            z_glmm_encoded_train = data_dict[f"z_glmm_encoded_train_{fold}"] 
            z_glmm_encoded_val = data_dict[f"z_glmm_encoded_val_{fold}"] 
            z_glmm_encoded_test = data_dict[f"z_glmm_encoded_test_{fold}"] 
            X_train_lr = pd.concat([X_train,z_glmm_encoded_train],axis=1)
            X_val_lr = pd.concat([X_val,z_glmm_encoded_val],axis=1)
            X_test_lr = pd.concat([X_test,z_glmm_encoded_test],axis=1)      
            X_train_val_lr = pd.concat([X_train_lr,X_val_lr])

            # Define data subset for XGB
            z_ordinal_encoded_train = data_dict[f"z_ordinal_encoded_train_{fold}"] 
            z_ordinal_encoded_val = data_dict[f"z_ordinal_encoded_val_{fold}"] 
            z_ordinal_encoded_test = data_dict[f"z_ordinal_encoded_test_{fold}"] 
            X_train_xgb = pd.concat([X_train,z_ordinal_encoded_train],axis=1)
            X_val_xgb = pd.concat([X_val,z_ordinal_encoded_val],axis=1)
            X_test_xgb = pd.concat([X_test,z_ordinal_encoded_test],axis=1)
            X_train_val_xgb = pd.concat([X_train_xgb,X_val_xgb])


            # Define data subset for evaluation
            X_train_val_lr = X_train_val_lr[[i for i in X_train_val_lr.columns if i in subsets[subset_key]]]
            X_test_lr = X_test_lr[[i for i in X_test_lr.columns if i in subsets[subset_key]]]
            X_train_val_xgb = X_train_val_xgb[[i for i in X_train_val_xgb.columns if i in subsets[subset_key]]]
            X_test_xgb = X_test_xgb[[i for i in X_test_xgb.columns if i in subsets[subset_key]]]


            # Train base models
            res, feats = evaluate_lr(X_train_val_lr, y_train_val, X_test_lr, y_test, target=target,tune=False, seed=RS)
            results_subsets[fold]["LR_"+subset_key] = res
            results_subsets_feature_importances[fold]["LR_"+subset_key] = feats

            res, feats = evaluate_xgb(X_train_val_xgb, y_train_val, X_test_xgb, y_test, target, tune=False, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_subsets[fold]["XGB_"+subset_key] = res
            results_subsets_feature_importances[fold]["XGB_"+subset_key] = feats

            # Train tuned models
            res, feats = evaluate_lr(X_train_val_lr, y_train_val, X_test_lr, y_test, target=target, max_evals=max_evals, tune=True, seed=RS)
            results_subsets[fold]["LR_"+subset_key+"_tuned"] = res
            results_subsets_feature_importances[fold]["LR_"+subset_key+"_tuned"] = feats

            res, feats = evaluate_xgb(X_train_val_xgb, y_train_val, X_test_xgb, y_test, target, tune=True, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_subsets[fold]["XGB_"+subset_key+"_tuned"] = res
            results_subsets_feature_importances[fold]["XGB_"+subset_key+"_tuned"] = feats
    
    if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
        os.makedirs(f"../results/{dataset_name}/{experiment_name}")
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle", 'wb') as handle:
        pickle.dump(results_subsets, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets_feature_importances.pickle", 'wb') as handle:
        pickle.dump(results_subsets_feature_importances, handle, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle", 'rb') as handle:
        results_subsets = pickle.load(handle)
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets_feature_importances.pickle", 'rb') as handle:
        results_subsets_feature_importances = pickle.load(handle)
        
        
results_subsets_df = pd.DataFrame(results_subsets[0]).transpose().sort_values("MSE Test",ascending=False).round(4)
results_subsets_df[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

Preparing results for fold 0, subset=demo_only
SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0054863557679075                                                                                              
SCORE: 0.9528008358900779                                                                                              
SCORE: 1.0072535959884104                                                                                              
SCORE: 1.0072535959884104                                                                                        

SCORE: 0.9945386052131653                                                                                              
SCORE: 0.9835792779922485                                                                                              
SCORE: 0.9573450088500977                                                                                              
SCORE: 0.9514812231063843                                                                                              
SCORE: 0.9720670580863953                                                                                              
SCORE: 0.9438234567642212                                                                                              
SCORE: 0.9387704730033875                                                                                              
SCORE: 0.9574514627456665                                                                                              
SCORE: 0.9404093623161316               

SCORE: 0.9736443758010864                                                                                              
SCORE: 0.9518270492553711                                                                                              
SCORE: 0.9619196057319641                                                                                              
SCORE: 1.0119068622589111                                                                                              
SCORE: 0.9396060109138489                                                                                              
SCORE: 0.968643069267273                                                                                               
SCORE: 0.9749679565429688                                                                                              
SCORE: 0.9406628608703613                                                                                              
SCORE: 0.9420291781425476               

SCORE: 0.9445825815200806                                                                                              
SCORE: 0.9310264587402344                                                                                              
SCORE: 0.9256736040115356                                                                                              
SCORE: 0.930517852306366                                                                                               
SCORE: 0.9301671981811523                                                                                              
100%|████████████████████████████████████████████████| 50/50 [00:18<00:00,  2.68trial/s, best loss: 0.9142577052116394]
The best hyperparameters after step 3 are :  

{'learning_rate': 0.1034242860381478, 'n_estimators': 110.0, 'seed': 0, 'max_depth': 4.0, 'min_child_weight': 2.0, 'colsample_bytree': 0.8858607179553077, 'subsample': 0.7832131396911284}
Test Performance after third tuning round: 0.

SCORE: 0.2762322425842285                                                                                              
SCORE: 0.17351481318473816                                                                                             
SCORE: 0.41830188035964966                                                                                             
SCORE: 0.20945677161216736                                                                                             
SCORE: 0.2154025137424469                                                                                              
SCORE: 0.3192102611064911                                                                                              
SCORE: 0.2259281575679779                                                                                              
SCORE: 0.24830758571624756                                                                                             
SCORE: 0.2231075018644333               

...

SCORE: 0.1867558606366992                                                                                              
SCORE: 0.1793369157329713                                                                                              
SCORE: 0.1663254687445173                                                                                              
SCORE: 0.16632742224774041                                                                                             
SCORE: 0.1745491736749904                                                                                              
SCORE: 0.17867778321465327                                                                                             
SCORE: 0.16930934784100027                                                                                             
SCORE: 0.17717751819194583                                                                                             
SCORE: 0.2243459254268881               

SCORE: 0.16137856245040894                                                                                             
SCORE: 0.15288624167442322                                                                                             
SCORE: 0.15600237250328064                                                                                             
SCORE: 0.15752510726451874                                                                                             
SCORE: 0.16802874207496643                                                                                             
SCORE: 0.1631356179714203                                                                                              
SCORE: 0.16004353761672974                                                                                             
SCORE: 0.1551283895969391                                                                                              
SCORE: 0.1506645381450653               

SCORE: 0.1798294633626938                                                                                              
100%|███████████████████████████████████████████████| 50/50 [00:11<00:00,  4.50trial/s, best loss: 0.13615567982196808]
The best hyperparameters after step 2 are :  

{'learning_rate': 0.48982433941567816, 'n_estimators': 114.0, 'seed': 0, 'max_depth': 3.0, 'min_child_weight': 0.0}
Test Performance after second tuning round: 0.25058525800704956
SCORE: 0.15415653586387634                                                                                             
SCORE: 0.17898093163967133                                                                                             
SCORE: 0.16226819157600403                                                                                             
SCORE: 0.16198208928108215                                                                                             
SCORE: 0.13977745175361633                           

SCORE: 0.6260214447975159                                                                                              
SCORE: 0.3419661819934845                                                                                              
SCORE: 0.4701296389102936                                                                                              
SCORE: 0.47653651237487793                                                                                             
SCORE: 0.6466430425643921                                                                                              
SCORE: 0.6396366953849792                                                                                              
SCORE: 0.45841288566589355                                                                                             
SCORE: 0.672563910484314                                                                                               
SCORE: 0.2730671465396881               

,MSE Train,R2 Train,MSE Test,R2 Test
XGB_demo_only,0.109700,0.890400,1.070700,-0.369900
XGB_activity_only,0.668600,0.332000,0.910900,-0.165400
XGB_demo_only_tuned,0.607100,0.393400,0.809000,-0.035000
Baseline,1.000900,-0.000100,0.781900,-0.000300
XGB_activity_only_tuned,0.776300,0.224300,0.778200,0.004400
LR_demo_only,0.898900,0.101800,0.768900,0.016300
LR_demo_only_tuned,0.906700,0.094000,0.765900,0.020100
XGB_activity_and_demo,0.026700,0.973300,0.755400,0.033600
LR_activity_only,0.914500,0.086200,0.734000,0.061000
LR_activity_and_demo,0.843700,0.156900,0.733300,0.061800


In [404]:
results_subsets_df = pd.DataFrame(results_subsets[0]).transpose().sort_values("MSE Test",ascending=False).round(4)
results_subsets_df[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,MSE Train,R2 Train,MSE Test,R2 Test
XGB_demo_only,0.109700,0.890400,1.070700,-0.369900
XGB_activity_only,0.668600,0.332000,0.910900,-0.165400
XGB_demo_only_tuned,0.607100,0.393400,0.809000,-0.035000
Baseline,1.000900,-0.000100,0.781900,-0.000300
XGB_activity_only_tuned,0.776300,0.224300,0.778200,0.004400
LR_demo_only,0.898900,0.101800,0.768900,0.016300
LR_demo_only_tuned,0.906700,0.094000,0.765900,0.020100
XGB_activity_and_demo,0.026700,0.973300,0.755400,0.033600
LR_activity_only,0.914500,0.086200,0.734000,0.061000
LR_activity_and_demo,0.843700,0.156900,0.733300,0.061800


### Effectiveness of Parameter Tuning

In [416]:
models = results_subsets[0].keys()
metric = "MSE Test"

subsets_folds_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())
subsets_mean_df = subsets_folds_df.mean(axis=0)
subsets_std_df = subsets_folds_df.std(axis=0)

methods = sorted(list(subsets_mean_df.index))[1:]
not_tuned = ["Baseline"]+methods[::2]
tuned = ["Baseline"]+methods[1::2]

res_df_tune_comp_mean = pd.DataFrame([subsets_mean_df.loc[not_tuned].values,subsets_mean_df.loc[tuned]],index=["Untuned","Tuned"],columns=not_tuned).transpose()
res_df_tune_comp_std = pd.DataFrame([subsets_std_df.loc[not_tuned].values,subsets_std_df.loc[tuned]],index=["Untuned","Tuned"],columns=not_tuned).transpose()

res_df_tune_comp_mean.round(2).style.highlight_min(color = 'lightgreen', axis = 1)

,Untuned,Tuned
Baseline,1.010000,1.010000
LR_activity_and_demo,0.900000,0.900000
LR_activity_only,0.930000,0.930000
LR_all,0.170000,0.160000
LR_demo_only,0.950000,0.950000
LR_performance_and_demo,0.170000,0.170000
LR_performance_only,0.170000,0.170000
XGB_activity_and_demo,0.910000,0.830000
XGB_activity_only,1.100000,0.920000
XGB_all,0.180000,0.130000


In [417]:
latex_df = res_df_tune_comp_mean.round(2).astype(str) + " (" +  res_df_tune_comp_std.round(3).astype(str) + ")"
latex_df

,Untuned,Tuned
Baseline,1.01 (0.175),1.01 (0.175)
LR_activity_and_demo,0.9 (0.165),0.9 (0.167)
LR_activity_only,0.93 (0.186),0.93 (0.186)
LR_all,0.17 (0.043),0.16 (0.042)
LR_demo_only,0.95 (0.149),0.95 (0.154)
LR_performance_and_demo,0.17 (0.046),0.17 (0.046)
LR_performance_only,0.17 (0.044),0.17 (0.044)
XGB_activity_and_demo,0.91 (0.183),0.83 (0.159)
XGB_activity_only,1.1 (0.186),0.92 (0.148)
XGB_all,0.18 (0.062),0.13 (0.033)


In [407]:
print(latex_df.to_latex())

\begin{tabular}{lll}
\toprule
{} &       Untuned &         Tuned \\
\midrule
Baseline                 &  1.01 (0.175) &  1.01 (0.175) \\
LR\_activity\_and\_demo     &   0.9 (0.165) &   0.9 (0.167) \\
LR\_activity\_only         &  0.93 (0.186) &  0.93 (0.186) \\
LR\_all                   &  0.17 (0.043) &  0.16 (0.042) \\
LR\_demo\_only             &  0.95 (0.149) &  0.95 (0.154) \\
LR\_performance\_and\_demo  &  0.17 (0.046) &  0.17 (0.046) \\
LR\_performance\_only      &  0.17 (0.044) &  0.17 (0.044) \\
XGB\_activity\_and\_demo    &  0.91 (0.183) &  0.83 (0.159) \\
XGB\_activity\_only        &   1.1 (0.186) &  0.92 (0.148) \\
XGB\_all                  &  0.18 (0.062) &  0.13 (0.033) \\
XGB\_demo\_only            &  1.09 (0.111) &  0.93 (0.148) \\
XGB\_performance\_and\_demo &  0.22 (0.043) &  0.18 (0.044) \\
XGB\_performance\_only     &  0.21 (0.037) &   0.19 (0.04) \\
\bottomrule
\end{tabular}



### Performance Results

In [408]:
# For LR
models = ["Baseline"]+[i for i in results_subsets[0].keys() if ("tuned" in i and "LR" in i)]
metric = "MSE Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())*-1

df_mean = pd.DataFrame((-1*use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_lr = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_lr.style.apply(negative_bold)


,Baseline,LR_demo_only_tuned,LR_performance_only_tuned,LR_activity_only_tuned,LR_activity_and_demo_tuned,LR_performance_and_demo_tuned,LR_all_tuned
0,1.01 (0.175),0.948 (0.154),0.17 (0.044),0.935 (0.186),0.897 (0.167),0.171 (0.046),0.164 (0.042)


In [409]:
# For XGB
models = ["Baseline"]+[i for i in results_subsets[0].keys() if ("tuned" in i and "XGB" in i)]
metric = "MSE Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())*-1

df_mean = pd.DataFrame((-1*use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_xgb = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_xgb.style.apply(negative_bold)


,Baseline,XGB_demo_only_tuned,XGB_performance_only_tuned,XGB_activity_only_tuned,XGB_activity_and_demo_tuned,XGB_performance_and_demo_tuned,XGB_all_tuned
0,1.01 (0.175),0.927 (0.148),0.192 (0.04),0.916 (0.148),0.825 (0.159),0.181 (0.044),0.135 (0.033)


In [410]:
res_df_lr.columns = [i[3:-6] if i != "Baseline" else "Baseline" for i in res_df_lr.columns]    
res_df_xgb.columns = [i[4:-6] if i != "Baseline" else "Baseline" for i in res_df_xgb.columns]    

latex_df_subsets = pd.concat([res_df_lr,res_df_xgb],axis=0)
latex_df_subsets.index = ["LR", "XGB"]
latex_df_subsets

,Baseline,demo_only,performance_only,activity_only,activity_and_demo,performance_and_demo,all
LR,1.01 (0.175),0.948 (0.154),0.17 (0.044),0.935 (0.186),0.897 (0.167),0.171 (0.046),0.164 (0.042)
XGB,1.01 (0.175),0.927 (0.148),0.192 (0.04),0.916 (0.148),0.825 (0.159),0.181 (0.044),0.135 (0.033)


In [411]:
print(latex_df_subsets.round(2).to_latex())


\begin{tabular}{llllllll}
\toprule
{} &      Baseline &      demo\_only & performance\_only &  activity\_only & activity\_and\_demo & performance\_and\_demo &            all \\
\midrule
LR  &  1.01 (0.175) &  0.948 (0.154) &     0.17 (0.044) &  0.935 (0.186) &     0.897 (0.167) &        0.171 (0.046) &  0.164 (0.042) \\
XGB &  1.01 (0.175) &  0.927 (0.148) &     0.192 (0.04) &  0.916 (0.148) &     0.825 (0.159) &        0.181 (0.044) &  0.135 (0.033) \\
\bottomrule
\end{tabular}



### Feature Importance

In [412]:
# top_10_importances = {}

# for model in list(results_subsets_feature_importances[fold].keys()):
#     imp_df = pd.concat([results_subsets_feature_importances[fold][model] for fold in range(folds)],axis=1)

#     if "LR" in model:
#         direction = imp_df.apply(lambda x: np.sign(x))
#         imp_df = imp_df.abs()

#     imp_df = imp_df/imp_df.sum(axis=0)

#     mean_imp_df = imp_df.mean(axis=1)
#     std_imp_df = imp_df.std(axis=1)

#     mean_imp_df = mean_imp_df.sort_values(ascending=False)
#     std_imp_df = std_imp_df.loc[mean_imp_df.index]
#     final_imps = mean_imp_df[:10]
#     final_imps["Rest"] = sum(mean_imp_df[10:])
#     top_5_importances[model] = np.array([final_imps.index.values, final_imps.values])

In [419]:
demo_importances = {}
demo_importances_stds = {}

for model in list(results_subsets_feature_importances[fold].keys()):
    if "demo" in model or "all" in model:
        imp_df_all = pd.concat([results_subsets_feature_importances[fold][model] for fold in range(folds)],axis=1)
        
        if "LR" in model:
            direction = imp_df_all.apply(lambda x: np.sign(x))
            imp_df_all = imp_df_all.abs()
        if imp_df_all.sum().sum()!=0:
            imp_df = imp_df_all/imp_df_all.sum(axis=0)
        imp_df = imp_df.fillna(1/imp_df.shape[0])
#         imp_df = imp_df.loc[demographic_cols]

#         mean_imp_df = imp_df.mean(axis=1)
#         std_imp_df = imp_df.std(axis=1)

#         mean_imp_df = mean_imp_df.sort_values(ascending=False)
#         std_imp_df = std_imp_df.loc[mean_imp_df.index]
#         final_imps = mean_imp_df#[:10]
#         final_imps["Rest"] = sum(mean_imp_df[10:])
#         final_imps["Total"] = sum(mean_imp_df)
        demo_importances[model] = np.round(np.mean(imp_df.loc[demographic_cols].sum(axis=0)),2)#final_imps.values
        demo_importances_stds[model] = np.round(np.std(imp_df.loc[demographic_cols].sum(axis=0)),2)#final_imps.values


In [420]:
lr_demo_imp = pd.Series({i: demo_importances[i] for i in demo_importances if "LR" in i and "tuned" in i})
xgb_demo_imp = pd.Series({i: demo_importances[i] for i in demo_importances if "XGB" in i and "tuned" in i})
lr_demo_imp.index = [i[3:-6] for i in lr_demo_imp.index]    
xgb_demo_imp.index = [i[4:-6] for i in xgb_demo_imp.index]    

lr_demo_imp_stds = pd.Series({i: demo_importances_stds[i] for i in demo_importances_stds if "LR" in i and "tuned" in i})
xgb_demo_imp_stds = pd.Series({i: demo_importances_stds[i] for i in demo_importances_stds if "XGB" in i and "tuned" in i})
lr_demo_imp_stds.index = [i[3:-6] for i in lr_demo_imp_stds.index]    
xgb_demo_imp_stds.index = [i[4:-6] for i in xgb_demo_imp_stds.index]    


latex_df_imp = pd.DataFrame([lr_demo_imp.astype(str) + " (" + lr_demo_imp_stds.astype(str) + ")",
                             xgb_demo_imp.astype(str) + " (" + xgb_demo_imp_stds.astype(str) + ")"])
latex_df_imp.index = ["LR", "XGB"]
latex_df_imp

,demo_only,activity_and_demo,performance_and_demo,all
LR,1.0 (0.0),0.5 (0.04),0.11 (0.08),0.03 (0.01)
XGB,1.0 (0.0),0.65 (0.03),0.17 (0.1),0.17 (0.09)


In [421]:
print(latex_df_subsets.to_latex())

\begin{tabular}{llllllll}
\toprule
{} &      Baseline &      demo\_only & performance\_only &  activity\_only & activity\_and\_demo & performance\_and\_demo &            all \\
\midrule
LR  &  1.01 (0.175) &  0.948 (0.154) &     0.17 (0.044) &  0.935 (0.186) &     0.897 (0.167) &        0.171 (0.046) &  0.164 (0.042) \\
XGB &  1.01 (0.175) &  0.927 (0.148) &     0.192 (0.04) &  0.916 (0.148) &     0.825 (0.159) &        0.181 (0.044) &  0.135 (0.033) \\
\bottomrule
\end{tabular}

